In [3]:
import requests
import json
import pandas as pd

In [4]:
# Fetch data from Eurostat API
url = "https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/tps00103?format=JSON&geoLevel=country&time=2012&time=2013&time=2014&time=2015&time=2016&time=2017&time=2018&time=2019&time=2020&time=2021&time=2022&time=2023"

response = requests.get(url)
data = response.json()


In [6]:
# Extract dimensions and values
values = data['value']
geo_dim = data['dimension']['geo']['category']
time_dim = data['dimension']['time']['category']

In [7]:
geo_codes = list(geo_dim['index'].keys())
time_codes = list(time_dim['index'].keys())
geo_labels = geo_dim['label']
num_years = len(time_codes)

In [8]:
# Transform to flat structure
transformed_data = []

for key, value in values.items():
    index = int(key)
    geo_idx = index // num_years
    time_idx = index % num_years

    geo_code = geo_codes[geo_idx]
    year = int(time_codes[time_idx])

    transformed_data.append({
        'geo': geo_code,
        'country': geo_labels.get(geo_code, geo_code),
        'year': year,
        'value': value
    })

# Create DataFrame
df = pd.DataFrame(transformed_data)

In [9]:
# Display first few rows
print(df.head(20))
print(f"\nTotal records: {len(df)}")
print(f"\nCountries: {df['geo'].nunique()}")
print(f"Years: {sorted(df['year'].unique())}")

# Save to CSV
df.to_csv('eurostat_pension_data.csv', index=False)
print("\nData saved to 'eurostat_pension_data.csv'")

# Save to JSON
df.to_json('eurostat_pension_data.json', orient='records', indent=2)
print("Data saved to 'eurostat_pension_data.json'")

# Optional: Create a pivot table for easier viewing
pivot_df = df.pivot(index='geo', columns='year', values='value')
print("\nPivot table (first 10 countries):")
print(pivot_df.head(10))

   geo                 country  year  value
0   AL                 Albania  2018   6.33
1   AL                 Albania  2019   6.50
2   AL                 Albania  2020   6.80
3   AL                 Albania  2021   6.72
4   AL                 Albania  2022   6.10
5   AT                 Austria  2012  14.50
6   AT                 Austria  2013  14.82
7   AT                 Austria  2014  14.90
8   AT                 Austria  2015  14.66
9   AT                 Austria  2016  14.36
10  AT                 Austria  2017  14.17
11  AT                 Austria  2018  14.01
12  AT                 Austria  2019  14.12
13  AT                 Austria  2020  15.38
14  AT                 Austria  2021  14.98
15  AT                 Austria  2022  14.19
16  AT                 Austria  2023  14.44
17  BA  Bosnia and Herzegovina  2013   9.61
18  BA  Bosnia and Herzegovina  2014   9.97
19  BA  Bosnia and Herzegovina  2015   9.70

Total records: 418

Countries: 37
Years: [np.int64(2012), np.int64(2013), n